In [1]:
!pip install optuna plotly matplotlib

In [2]:
import optuna
import plotly.io as pio

# Optional: Set a default Plotly template for nicer plots in Jupyter
pio.templates.default = "plotly_white"

# --- 1. Define Study Database ---
# Assumes 'optima_studies.db' is in the same directory as your Jupyter Notebook.
# If it's elsewhere, provide the full or relative path.
db_file = "sqlite:///optima_studies.db"

# --- 2. List Available Studies (Optional but Recommended) ---
try:
    study_summaries = optuna.study.get_all_study_summaries(storage=db_file)
    print("Available studies in the database:")
    if not study_summaries:
        print("  No studies found in the database.")
    for i, summary in enumerate(study_summaries):
        print(f"  {i}: Study name: '{summary.study_name}', Trials: {summary.n_trials}")
except Exception as e:
    print(f"Error accessing studies from {db_file}: {e}")
    print("Please ensure 'optima_studies.db' exists and is a valid Optuna database.")
    # You might want to stop execution here if studies can't be listed.
    raise

# --- 3. Load a Specific Study ---
# Replace 'YOUR_STUDY_NAME_HERE' with the actual name of the study you want to analyze.
# You can get the name from the list printed above.
# For example, if your study was named 'v4.1-hpo-prod' from previous discussions:
study_name_to_load = 'YOUR_STUDY_NAME_HERE' # <<< --- IMPORTANT: SET YOUR STUDY NAME HERE

print(f"\nAttempting to load study: '{study_name_to_load}'")
try:
    study = optuna.load_study(study_name=study_name_to_load, storage=db_file)
    print(f"Successfully loaded study '{study.study_name}'.")
except KeyError:
    print(f"Error: Study with name '{study_name_to_load}' not found in the database.")
    print("Please check the study name and the list of available studies above.")
    # You might want to stop execution here if the study isn't found.
    raise
except Exception as e:
    print(f"An unexpected error occurred while loading the study: {e}")
    raise


# --- 4. Display Basic Study Information ---
if 'study' in locals() and study.trials: # Check if study loaded and has trials
    print("\n--- Study Summary ---")
    print(f"  Number of finished trials: {len([t for t in study.trials if t.state == optuna.trial.TrialState.COMPLETE])}")
    
    best_trial = study.best_trial
    print(f"  Best trial number: {best_trial.number}")
    print(f"  Best value (objective): {best_trial.value:.6f}") # Adjust formatting as needed
    print("  Best parameters found:")
    for key, value in best_trial.params.items():
        if isinstance(value, float):
            print(f"    {key}: {value:.6f}") # Adjust formatting
        else:
            print(f"    {key}: {value}")
    
    # Get parameter names for use in plots later
    param_names = list(best_trial.params.keys())
    print(f"  Parameter names in this study: {param_names}")
else:
    if 'study' in locals():
        print("\nStudy loaded, but it appears to have no completed trials to analyze.")

# --- 5. Generate and Display Optuna Visualizations ---
# These plots will be displayed directly in your Jupyter Notebook output cell.
if 'study' in locals() and any(t.state == optuna.trial.TrialState.COMPLETE for t in study.trials):
    print("\n--- Generating Visualizations ---")

    # a) Optimization History
    # Shows how the best objective value improved over trials.
    print("\n1. Optimization History Plot:")
    fig_history = optuna.visualization.plot_optimization_history(study)
    fig_history.show()

    # b) Parameter Importances
    # Ranks parameters by their influence on the objective value.
    # This requires either that importances were computed during the study (e.g. via a callback)
    # or that Optuna can compute them now (e.g., using a surrogate model if enough trials exist).
    # If this plot is empty or errors, it might be due to the study setup or number of trials.
    print("\n2. Parameter Importance Plot:")
    try:
        fig_importance = optuna.visualization.plot_param_importances(study)
        fig_importance.show()
    except Exception as e:
        print(f"  Could not generate parameter importance plot. Error: {e}")
        print("  This can happen if importances were not logged or cannot be computed (e.g., too few trials or specific samplers).")

    # c) Parallel Coordinate Plot
    # Good for visualizing relationships between multiple parameters and the objective.
    # You can optionally restrict the parameters shown by passing a list to the `params` argument.
    print("\n3. Parallel Coordinate Plot (showing all parameters):")
    # Example: fig_parallel = optuna.visualization.plot_parallel_coordinate(study, params=['d', 'L', 'dropout'])
    fig_parallel = optuna.visualization.plot_parallel_coordinate(study) 
    fig_parallel.show()

    # d) Slice Plot
    # Shows how the objective value changes as individual parameters are varied.
    # Useful for seeing the optimal range for each parameter.
    # Plots for all parameters by default. You can select specific ones: params=['d', 'L']
    if param_names: # Check if we have parameter names
        print("\n4. Slice Plots (showing for all parameters):")
        # Example: fig_slice = optuna.visualization.plot_slice(study, params=['d', 'L'])
        fig_slice = optuna.visualization.plot_slice(study) 
        fig_slice.show()
    else:
        print("\n  Skipping slice plots as no parameter names were identified (e.g., no completed trials).")

    # e) Contour Plot
    # Shows relationships between pairs of parameters and the objective.
    # You need to specify which pairs of parameters to plot.
    # Let's try to plot for the first two parameters if at least two exist.
    if len(param_names) >= 2:
        print(f"\n5. Contour Plot (example for parameters: '{param_names[0]}' and '{param_names[1]}'):")
        # Example: fig_contour = optuna.visualization.plot_contour(study, params=['d', 'L'])
        try:
            fig_contour = optuna.visualization.plot_contour(study, params=[param_names[0], param_names[1]])
            fig_contour.show()
        except Exception as e:
            print(f"  Could not generate contour plot for '{param_names[0]}' and '{param_names[1]}'. Error: {e}")
            print("  This can happen for various reasons, e.g., parameter types or distributions.")

        # If you have more parameters, you can generate more contour plots:
        # if len(param_names) >= 3:
        #    print(f"\n   Contour Plot (example for parameters: '{param_names[0]}' and '{param_names[2]}'):")
        #    fig_contour_2 = optuna.visualization.plot_contour(study, params=[param_names[0], param_names[2]])
        #    fig_contour_2.show()
    else:
        print("\n  Skipping contour plots as fewer than two parameters were identified.")
        
    print("\n--- End of Visualizations ---")
    print("Note: If plots appear small or crowded, you can often interact with them (zoom, pan) if using Plotly.")
    print("You can also save them to files, e.g., fig_history.write_image('history.png') or fig_history.write_html('history.html')")

else:
    if 'study' in locals():
        print("\nNo completed trials in the study to visualize.")

Available studies in the database:
  No studies found in the database.

Attempting to load study: 'YOUR_STUDY_NAME_HERE'
Error: Study with name 'YOUR_STUDY_NAME_HERE' not found in the database.
Please check the study name and the list of available studies above.


KeyError: 'Record does not exist.'